# 03_data_matching.ipynb - Матчинг готових даних

## 🔗 Зіставлення push-користувачів з конверсіями

Цей ноутбук завантажує готові дані з попередніх ноутбуків та виконує їх матчинг по GADID.

### Вхідні дані:
- **`push_users_data.parquet`** - 3,132,011 користувачів з push-даними (з 01_eda_statistic)
- **`conversion_users_data.parquet`** - 166,103 користувачів з конверсіями (з 02_eda_keitaro)

### Результат:
- Матчинг по GADID
- Підготовка даних для A/B та гео аналізу

---

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🔗 МАТЧИНГ ГОТОВИХ ДАНИХ")
print("=" * 50)
print(f"📂 Завантажуємо збережені файли з попередніх ноутбуків")
print(f"🎯 Мета: Швидкий матчинг без повторних запитів до БД")

🔗 МАТЧИНГ ГОТОВИХ ДАНИХ
📂 Завантажуємо збережені файли з попередніх ноутбуків
🎯 Мета: Швидкий матчинг без повторних запитів до БД


## **КРОК 1: Перевірка наявності файлів**

In [2]:
print("\n🔗 КРОК 1: ПЕРЕВІРКА НАЯВНОСТІ ФАЙЛІВ")
print("-" * 40)

# Перевіряємо чи існують файли
files_to_check = {
    'push_data': '../data/processed/push_users_data.parquet',
    'conversion_data': '../data/processed/conversion_users_data.parquet',
    'push_summary': '../outputs/tables/push_summary_stats.json',
    'conversion_summary': '../outputs/tables/conversion_summary_stats.json'
}

available_files = {}
missing_files = []

for name, path in files_to_check.items():
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / 1024 / 1024
        available_files[name] = path
        print(f"✅ {name}: {path} ({size_mb:.1f} MB)")
    else:
        missing_files.append(name)
        print(f"❌ {name}: {path} - файл не знайдено")

if missing_files:
    print(f"\n⚠️ Відсутні файли: {missing_files}")
    print(f"💡 Запустіть спочатку ноутбуки 01_eda_statistic та 02_eda_keitaro")
    raise FileNotFoundError(f"Відсутні необхідні файли: {missing_files}")
else:
    print(f"\n✅ Всі необхідні файли знайдено!")


🔗 КРОК 1: ПЕРЕВІРКА НАЯВНОСТІ ФАЙЛІВ
----------------------------------------
✅ push_data: ../data/processed/push_users_data.parquet (132.8 MB)
✅ conversion_data: ../data/processed/conversion_users_data.parquet (9.3 MB)
✅ push_summary: ../outputs/tables/push_summary_stats.json (0.0 MB)
✅ conversion_summary: ../outputs/tables/conversion_summary_stats.json (0.0 MB)

✅ Всі необхідні файли знайдено!


## **КРОК 2: Завантаження push-даних**

In [3]:
print("\n🔗 КРОК 2: ЗАВАНТАЖЕННЯ PUSH-ДАНИХ")
print("-" * 40)

# Завантажуємо готові push-дані
print("📂 Завантаження push_users_data.parquet...")
push_df = pd.read_parquet('../data/processed/push_users_data.parquet')

print(f"✅ Push-дані завантажено: {len(push_df):,} записів")
print(f"📊 Колонки: {list(push_df.columns)}")

# Перевіряємо структуру даних
print("\n📈 СТРУКТУРА PUSH-ДАНИХ:")
print(f"   📱 Унікальних користувачів (gadid): {push_df['gadid'].nunique():,}")
print(f"   📱 Загальна к-ть push: {push_df['total_pushes'].sum():,}")
print(f"   📱 Середня к-ть push: {push_df['total_pushes'].mean():.2f}")
print(f"   📱 Медіанна к-ть push: {push_df['total_pushes'].median():.0f}")

if 'ab_group' in push_df.columns:
    print(f"   🏷️ A/B групи: {push_df['ab_group'].value_counts().to_dict()}")

if 'tier' in push_df.columns:
    print(f"   🌍 Розподіл по tier: {push_df['tier'].value_counts().to_dict()}")
elif 'country' in push_df.columns:
    print(f"   🌍 Топ-5 країн: {push_df['country'].value_counts().head().to_dict()}")

# Показуємо приклад
print("\n🔍 ПРИКЛАД PUSH-ДАНИХ:")
display(push_df.head())

# Завантажуємо summary
if os.path.exists('../outputs/tables/push_summary_stats.json'):
    with open('../outputs/tables/push_summary_stats.json', 'r', encoding='utf-8') as f:
        push_summary = json.load(f)
    print(f"\n📊 SUMMARY З 01_EDA_STATISTIC:")
    print(f"   📅 Період: {push_summary.get('period', 'N/A')}")
    print(f"   👥 Унікальних користувачів: {push_summary.get('unique_users', 'N/A'):,}")
    print(f"   📱 A/B груп: {push_summary.get('unique_ab_groups', 'N/A')}")
    print(f"   🌍 Країн: {push_summary.get('unique_countries', 'N/A')}")


🔗 КРОК 2: ЗАВАНТАЖЕННЯ PUSH-ДАНИХ
----------------------------------------
📂 Завантаження push_users_data.parquet...
✅ Push-дані завантажено: 3,132,011 записів
📊 Колонки: ['gadid', 'ab_group', 'country', 'total_pushes', 'successful_pushes', 'success_rate', 'first_push', 'last_push', 'active_days', 'tier', 'campaign_duration_hours', 'pushes_per_day']

📈 СТРУКТУРА PUSH-ДАНИХ:
   📱 Унікальних користувачів (gadid): 2,824,736
   📱 Загальна к-ть push: 69,964,082
   📱 Середня к-ть push: 22.34
   📱 Медіанна к-ть push: 22
   🏷️ A/B групи: {'5': 628265, '2': 626824, '4': 626170, '3': 625849, '1': 624879, '6': 24}
   🌍 Розподіл по tier: {'Unknown': 1749414, 'Tier 2': 679902, 'Tier 3': 301230, 'Other': 265848, 'Tier 1': 135617}

🔍 ПРИКЛАД PUSH-ДАНИХ:


,gadid,ab_group,country,total_pushes,successful_pushes,success_rate,first_push,last_push,active_days,tier,campaign_duration_hours,pushes_per_day
0,00000000-0000-0000-0000-000000000000,5,Unknown,339118,39195,0.1156,2025-05-22 00:03:58+00:00,2025-05-29 23:38:31+00:00,8,Unknown,191.575833,42389.75
1,00000000-0000-0000-0000-000000000000,4,Unknown,281586,33174,0.1178,2025-05-22 00:03:58+00:00,2025-05-29 23:38:31+00:00,8,Unknown,191.575833,35198.25
2,00000000-0000-0000-0000-000000000000,3,Unknown,213798,25603,0.1198,2025-05-22 00:03:58+00:00,2025-05-29 23:38:32+00:00,8,Unknown,191.576111,26724.75
3,00000000-0000-0000-0000-000000000000,2,Unknown,144698,17209,0.1189,2025-05-22 00:03:58+00:00,2025-05-29 23:38:28+00:00,8,Unknown,191.575000,18087.25
4,00000000-0000-0000-0000-000000000000,1,Unknown,73983,9338,0.1262,2025-05-22 00:04:08+00:00,2025-05-29 23:38:24+00:00,8,Unknown,191.571111,9247.88



📊 SUMMARY З 01_EDA_STATISTIC:
   📅 Період: 2025-05-22 - 2025-05-29
   👥 Унікальних користувачів: 2,824,736
   📱 A/B груп: 6
   🌍 Країн: 208


## **КРОК 3: Завантаження конверсійних даних**

In [4]:
print("\n🔗 КРОК 3: ЗАВАНТАЖЕННЯ КОНВЕРСІЙНИХ ДАНИХ")
print("-" * 40)

# Завантажуємо готові конверсійні дані
print("📂 Завантаження conversion_users_data.parquet...")
conversion_df = pd.read_parquet('../data/processed/conversion_users_data.parquet')

print(f"✅ Конверсійні дані завантажено: {len(conversion_df):,} записів")
print(f"📊 Колонки: {list(conversion_df.columns)}")

# Перевіряємо структуру
print("\n📈 СТРУКТУРА КОНВЕРСІЙНИХ ДАНИХ:")
print(f"   👥 Унікальних користувачів (gadid): {conversion_df['gadid'].nunique():,}")

if 'total_deposits' in conversion_df.columns:
    deposit_users = (conversion_df['total_deposits'] > 0).sum()
    print(f"   💰 Користувачі з депозитами: {deposit_users:,}")
    print(f"   💰 Загальна сума депозитів: {conversion_df['total_deposits'].sum():,}")

if 'total_registrations' in conversion_df.columns:
    reg_users = (conversion_df['total_registrations'] > 0).sum()
    print(f"   📝 Користувачі з реєстраціями: {reg_users:,}")
    print(f"   📝 Загальна к-ть реєстрацій: {conversion_df['total_registrations'].sum():,}")

if 'total_revenue' in conversion_df.columns:
    total_revenue = conversion_df['total_revenue'].sum()
    print(f"   💵 Загальний дохід: ${total_revenue:,.2f}")
    print(f"   💵 ARPU: ${total_revenue/len(conversion_df):.2f}")

if 'tier' in conversion_df.columns:
    print(f"   🌍 Розподіл по tier: {conversion_df['tier'].value_counts().to_dict()}")

# TMNT групи якщо є
if 'group_name' in conversion_df.columns:
    print(f"   🐢 TMNT групи: {conversion_df['group_name'].value_counts().to_dict()}")

# Показуємо приклад
print("\n🔍 ПРИКЛАД КОНВЕРСІЙНИХ ДАНИХ:")
display(conversion_df.head())

# Завантажуємо summary
if os.path.exists('../outputs/tables/conversion_summary_stats.json'):
    with open('../outputs/tables/conversion_summary_stats.json', 'r', encoding='utf-8') as f:
        conv_summary = json.load(f)
    print(f"\n📊 SUMMARY З 02_EDA_KEITARO:")
    if 'analysis_info' in conv_summary:
        print(f"   📅 Період: {conv_summary['analysis_info'].get('period', 'N/A')}")
        print(f"   🎯 Цільові застосунки: {conv_summary['analysis_info'].get('target_apps', [])}")
    if 'revenue_metrics' in conv_summary:
        print(f"   💵 Загальний дохід: ${conv_summary['revenue_metrics'].get('total_revenue', 0):,.2f}")
        print(f"   💵 Середній ARPU: ${conv_summary['revenue_metrics'].get('avg_revenue_per_user', 0):.2f}")
    if 'conversion_metrics' in conv_summary:
        print(f"   🎖️ Коефіцієнт депозитів: {conv_summary['conversion_metrics'].get('deposit_rate', 0):.1f}%")


🔗 КРОК 3: ЗАВАНТАЖЕННЯ КОНВЕРСІЙНИХ ДАНИХ
----------------------------------------
📂 Завантаження conversion_users_data.parquet...
✅ Конверсійні дані завантажено: 166,103 записів
📊 Колонки: ['gadid', 'country', 'campaign_id', 'group_id', 'group_name', 'total_deposits', 'total_registrations', 'total_revenue', 'lead_revenue', 'total_events', 'first_conversion_date', 'last_conversion_date', 'first_conversion_datetime', 'last_conversion_datetime', 'conversion_days', 'conversion_type', 'tier', 'conversion_window_days', 'avg_revenue_per_conversion', 'arpu']

📈 СТРУКТУРА КОНВЕРСІЙНИХ ДАНИХ:
   👥 Унікальних користувачів (gadid): 163,788
   💰 Користувачі з депозитами: 24,503
   💰 Загальна сума депозитів: 24,549
   📝 Користувачі з реєстраціями: 141,878
   📝 Загальна к-ть реєстрацій: 142,649
   💵 Загальний дохід: $245,628.07
   💵 ARPU: $1.48
   🌍 Розподіл по tier: {'Tier 2': 104517, 'Tier 3': 29026, 'Other': 20909, 'Tier 1': 11429, 'Unknown': 222}
   🐢 TMNT групи: {'Splinter (Oleksandr)': 67184,

,gadid,country,campaign_id,group_id,group_name,total_deposits,total_registrations,total_revenue,lead_revenue,total_events,first_conversion_date,last_conversion_date,first_conversion_datetime,last_conversion_datetime,conversion_days,conversion_type,tier,conversion_window_days,avg_revenue_per_conversion,arpu
0,94fd2965-34b3-409e-b5ab-3a8372509b44,KZ,2879,1038,Zhenya (Leonardo),1,0,12512.00,0.0,1,2025-05-31,2025-05-31,2025-05-31 19:45:10,2025-05-31 19:45:10,1,Deposit,Other,1,12512.00,12512.00
1,e114b39d-f2c9-469e-ba69-7465f57624a8,LT,2884,1039,Bohdan (Raphael),3,0,460.00,0.0,3,2025-05-25,2025-05-25,2025-05-25 19:15:47,2025-05-25 19:15:47,1,Deposit,Tier 2,1,153.33,460.00
2,57d09d48-fba3-42e6-ac36-3765f83159de,LT,2860,1065,Splinter (Oleksandr),3,0,460.00,0.0,3,2025-05-25,2025-05-25,2025-05-25 10:13:48,2025-05-25 10:13:48,1,Deposit,Tier 2,1,153.33,460.00
3,e671ca48-9655-44d1-99fb-e7fa666a2244,ES,2860,1065,Splinter (Oleksandr),2,0,363.16,0.0,2,2025-05-25,2025-05-30,2025-05-25 16:57:53,2025-05-30 21:02:21,2,Deposit,Tier 2,6,181.58,363.16
4,afd104fb-57dd-4ec6-a1cc-6dff44d2636a,ES,2826,1038,Zhenya (Leonardo),2,0,307.16,0.0,2,2025-05-26,2025-05-27,2025-05-26 21:55:49,2025-05-27 00:02:18,2,Deposit,Tier 2,2,153.58,307.16



📊 SUMMARY З 02_EDA_KEITARO:
   📅 Період: 2025-05-22 - 2025-06-07
   🎯 Цільові застосунки: ['Michelangelo', 'Leonardo', 'Raphael', 'Splinter']
   💵 Загальний дохід: $245,628.07
   💵 Середній ARPU: $1.48
   🎖️ Коефіцієнт депозитів: 14.8%


## **КРОК 4: Матчинг даних по GADID**

In [5]:
print("\n🔗 КРОК 4: МАТЧИНГ ПО GADID")
print("-" * 40)

# Аналіз пересічення множин
push_gadids = set(push_df['gadid'])
conv_gadids = set(conversion_df['gadid'])

intersection = push_gadids & conv_gadids
push_only = push_gadids - conv_gadids
conv_only = conv_gadids - push_gadids

print(f"📊 АНАЛІЗ ПЕРЕСІЧЕННЯ GADID:")
print(f"   📱 Користувачі лише з push: {len(push_only):,}")
print(f"   💰 Користувачі лише з конверсіями: {len(conv_only):,}")
print(f"   🔗 Користувачі з обома типами даних: {len(intersection):,}")
print(f"   📈 % покриття (від push): {len(intersection)/len(push_gadids)*100:.2f}%")
print(f"   📈 % покриття (від конверсій): {len(intersection)/len(conv_gadids)*100:.2f}%")

# Виконуємо LEFT JOIN - всі push користувачі + їх конверсії
print(f"\n🔗 Виконання LEFT JOIN матчингу...")
matched_df = push_df.merge(
    conversion_df, 
    on='gadid', 
    how='left',
    suffixes=('_push', '_conv')
)

print(f"✅ Матчинг завершено: {len(matched_df):,} записів")

# Обробляємо пропуски
numeric_cols = ['total_deposits', 'total_registrations', 'total_revenue']
for col in numeric_cols:
    if col in matched_df.columns:
        matched_df[col] = matched_df[col].fillna(0)

# Створюємо бінарні індикатори
if 'total_deposits' in matched_df.columns:
    matched_df['has_deposit'] = (matched_df['total_deposits'] > 0).astype(int)
if 'total_registrations' in matched_df.columns:
    matched_df['has_registration'] = (matched_df['total_registrations'] > 0).astype(int)

matched_df['has_conversion'] = 0
if 'total_deposits' in matched_df.columns and 'total_registrations' in matched_df.columns:
    matched_df['has_conversion'] = ((matched_df['total_deposits'] > 0) | (matched_df['total_registrations'] > 0)).astype(int)

# Використовуємо найкращі дані по країнах та tier
if 'country_push' in matched_df.columns and 'country_conv' in matched_df.columns:
    matched_df['country_final'] = matched_df['country_push'].fillna(matched_df['country_conv'])
elif 'country' in matched_df.columns:
    matched_df['country_final'] = matched_df['country']

if 'tier_push' in matched_df.columns and 'tier_conv' in matched_df.columns:
    matched_df['tier_final'] = matched_df['tier_push'].fillna(matched_df['tier_conv'])
elif 'tier' in matched_df.columns:
    matched_df['tier_final'] = matched_df['tier']

print(f"\n🎯 РЕЗУЛЬТАТ МАТЧИНГУ:")
print(f"   👥 Загалом користувачів: {len(matched_df):,}")
if 'has_conversion' in matched_df.columns:
    conv_count = matched_df['has_conversion'].sum()
    conv_rate = matched_df['has_conversion'].mean() * 100
    print(f"   ✅ З конверсіями: {conv_count:,} ({conv_rate:.2f}%)")
if 'has_deposit' in matched_df.columns:
    dep_count = matched_df['has_deposit'].sum()
    dep_rate = matched_df['has_deposit'].mean() * 100
    print(f"   💰 З депозитами: {dep_count:,} ({dep_rate:.2f}%)")
if 'has_registration' in matched_df.columns:
    reg_count = matched_df['has_registration'].sum()
    reg_rate = matched_df['has_registration'].mean() * 100
    print(f"   📝 З реєстраціями: {reg_count:,} ({reg_rate:.2f}%)")

# Показуємо приклад результату
print("\n🔍 ПРИКЛАД МАТЧИНГУ:")
display_cols = ['gadid', 'total_pushes', 'ab_group']
if 'has_deposit' in matched_df.columns:
    display_cols.append('has_deposit')
if 'total_revenue' in matched_df.columns:
    display_cols.append('total_revenue')
if 'tier_final' in matched_df.columns:
    display_cols.append('tier_final')

display(matched_df[display_cols].head(10))


🔗 КРОК 4: МАТЧИНГ ПО GADID
----------------------------------------
📊 АНАЛІЗ ПЕРЕСІЧЕННЯ GADID:
   📱 Користувачі лише з push: 2,714,878
   💰 Користувачі лише з конверсіями: 53,930
   🔗 Користувачі з обома типами даних: 109,858
   📈 % покриття (від push): 3.89%
   📈 % покриття (від конверсій): 67.07%

🔗 Виконання LEFT JOIN матчингу...
✅ Матчинг завершено: 3,219,484 записів

🎯 РЕЗУЛЬТАТ МАТЧИНГУ:
   👥 Загалом користувачів: 3,219,484
   ✅ З конверсіями: 217,499 (6.76%)
   💰 З депозитами: 41,193 (1.28%)
   📝 З реєстраціями: 187,205 (5.81%)

🔍 ПРИКЛАД МАТЧИНГУ:


,gadid,total_pushes,ab_group,has_deposit,total_revenue,tier_final
0,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
1,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
2,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
3,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
4,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
5,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
6,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
7,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
8,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown
9,00000000-0000-0000-0000-000000000000,339118,5,1,0.0,Unknown


## **КРОК 5: Додаткові метрики та категоризація**

In [6]:
print("\n🔗 КРОК 5: ДОДАТКОВІ МЕТРИКИ")
print("-" * 40)

# Додаємо корисні метрики
if 'total_pushes' in matched_df.columns and 'total_revenue' in matched_df.columns:
    matched_df['revenue_per_push'] = matched_df['total_revenue'] / matched_df['total_pushes'].clip(lower=1)
    print("✅ Додано revenue_per_push")

# Категорії користувачів
def categorize_user(row):
    if 'has_deposit' in row and row['has_deposit'] > 0:
        return 'Depositor'
    elif 'has_registration' in row and row['has_registration'] > 0:
        return 'Registration Only'
    else:
        return 'No Conversion'

matched_df['user_category'] = matched_df.apply(categorize_user, axis=1)
print("✅ Додано user_category")

# Сегменти по кількості push-ів
if 'total_pushes' in matched_df.columns:
    matched_df['push_segment'] = pd.cut(
        matched_df['total_pushes'],
        bins=[0, 1, 5, 10, 20, 50, float('inf')],
        labels=['1', '2-5', '6-10', '11-20', '21-50', '50+']
    )
    print("✅ Додано push_segment")

# Статистика по категоріях
print("\n👥 РОЗПОДІЛ ПО КАТЕГОРІЯХ:")
category_stats = matched_df['user_category'].value_counts()
category_pct = matched_df['user_category'].value_counts(normalize=True) * 100

for category in category_stats.index:
    count = category_stats[category]
    pct = category_pct[category]
    print(f"   {category}: {count:,} ({pct:.1f}%)")

# A/B аналіз якщо є групи
if 'ab_group' in matched_df.columns:
    print("\n🏷️ ШВИДКИЙ A/B АНАЛІЗ:")
    ab_stats = matched_df.groupby('ab_group').agg({
        'gadid': 'count',
        'total_pushes': 'mean'
    }).round(2)
    
    if 'has_deposit' in matched_df.columns:
        ab_stats['deposits'] = matched_df.groupby('ab_group')['has_deposit'].sum()
        ab_stats['deposit_rate'] = (ab_stats['deposits'] / ab_stats['gadid'] * 100).round(2)
    
    if 'total_revenue' in matched_df.columns:
        ab_stats['total_revenue'] = matched_df.groupby('ab_group')['total_revenue'].sum().round(2)
    
    print(ab_stats)

# Tier аналіз якщо є
if 'tier_final' in matched_df.columns:
    print("\n🌍 ШВИДКИЙ TIER АНАЛІЗ:")
    tier_stats = matched_df.groupby('tier_final').agg({
        'gadid': 'count',
        'total_pushes': 'mean'
    }).round(2)
    
    if 'has_deposit' in matched_df.columns:
        tier_stats['deposits'] = matched_df.groupby('tier_final')['has_deposit'].sum()
        tier_stats['deposit_rate'] = (tier_stats['deposits'] / tier_stats['gadid'] * 100).round(2)
    
    print(tier_stats)


🔗 КРОК 5: ДОДАТКОВІ МЕТРИКИ
----------------------------------------
✅ Додано revenue_per_push
✅ Додано user_category
✅ Додано push_segment

👥 РОЗПОДІЛ ПО КАТЕГОРІЯХ:
   No Conversion: 3,001,985 (93.2%)
   Registration Only: 176,306 (5.5%)
   Depositor: 41,193 (1.3%)

🏷️ ШВИДКИЙ A/B АНАЛІЗ:
           gadid  total_pushes  deposits  deposit_rate  total_revenue
ab_group                                                             
1         643636         39.28      8386          1.30       40881.45
2         644463         76.58      8304          1.29       44058.78
3         643002        114.97      8168          1.27       47040.36
4         642431        150.13      7928          1.23       50624.46
5         645896        181.13      8382          1.30       44210.23
6             56         24.98        25         44.64           0.00

🌍 ШВИДКИЙ TIER АНАЛІЗ:
              gadid  total_pushes  deposits  deposit_rate
tier_final                                               
Other  

## **КРОК 6: Збереження результатів матчингу**

In [7]:
print("\n🔗 КРОК 6: ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ")
print("-" * 40)

# Створюємо папки
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../outputs/tables', exist_ok=True)

# Основний файл матчингу
matched_df.to_parquet('../data/processed/matched_push_conversions.parquet', index=False)
print("✅ Основний файл: matched_push_conversions.parquet")

# Файл для A/B аналізу
if 'ab_group' in matched_df.columns:
    ab_ready = matched_df[[
        'gadid', 'ab_group', 'total_pushes', 
        'has_deposit', 'has_registration', 'total_revenue', 
        'tier_final', 'user_category', 'push_segment'
    ]].copy()
    ab_ready.to_parquet('../data/processed/ab_analysis_ready.parquet', index=False)
    ab_ready.to_csv('../outputs/tables/ab_analysis_ready.csv', index=False)
    print("✅ Файли для A/B аналізу збережено")

# Файл для гео аналізу
if 'tier_final' in matched_df.columns:
    geo_ready = matched_df[[
        'gadid', 'tier_final', 'country_final', 'total_pushes',
        'has_deposit', 'total_revenue', 'ab_group', 'user_category'
    ]].copy()
    geo_ready.to_parquet('../data/processed/geo_analysis_ready.parquet', index=False)
    geo_ready.to_csv('../outputs/tables/geo_analysis_ready.csv', index=False)
    print("✅ Файли для гео аналізу збережено")

# Резюме матчингу
matching_summary = {
    'matching_info': {
        'push_users': len(push_df),
        'conversion_users': len(conversion_df),
        'matched_users': len(matched_df),
        'overlapping_users': len(intersection),
        'overlap_rate_from_push': len(intersection) / len(push_gadids) * 100,
        'overlap_rate_from_conversions': len(intersection) / len(conv_gadids) * 100,
        'matching_quality': 'excellent' if len(intersection) > 10000 else ('good' if len(intersection) > 1000 else 'poor')
    },
    'final_metrics': {
        'total_users': len(matched_df),
        'users_with_deposits': int(matched_df['has_deposit'].sum()) if 'has_deposit' in matched_df.columns else 0,
        'users_with_registrations': int(matched_df['has_registration'].sum()) if 'has_registration' in matched_df.columns else 0,
        'deposit_conversion_rate': float(matched_df['has_deposit'].mean() * 100) if 'has_deposit' in matched_df.columns else 0,
        'total_revenue': float(matched_df['total_revenue'].sum()) if 'total_revenue' in matched_df.columns else 0,
        'arpu': float(matched_df['total_revenue'].sum() / len(matched_df)) if 'total_revenue' in matched_df.columns else 0
    },
    'segmentation': {
        'user_categories': matched_df['user_category'].value_counts().to_dict(),
        'ab_groups': matched_df['ab_group'].value_counts().to_dict() if 'ab_group' in matched_df.columns else {},
        'tiers': matched_df['tier_final'].value_counts().to_dict() if 'tier_final' in matched_df.columns else {}
    },
    'processing_info': {
        'completed_at': datetime.now().isoformat(),
        'source_files': {
            'push_data': 'push_users_data.parquet',
            'conversion_data': 'conversion_users_data.parquet'
        }
    }
}

# Зберігаємо резюме
with open('../outputs/tables/matching_summary.json', 'w', encoding='utf-8') as f:
    json.dump(matching_summary, f, ensure_ascii=False, indent=2)

print("✅ Резюме матчингу збережено: matching_summary.json")

# Фінальна статистика
print(f"\n📈 ФІНАЛЬНА СТАТИСТИКА МАТЧИНГУ:")
print(f"   👥 Загалом користувачів: {len(matched_df):,}")
print(f"   🔗 Спільних GADID: {len(intersection):,}")
print(f"   📈 Якість матчингу: {matching_summary['matching_info']['matching_quality']}")
if 'has_deposit' in matched_df.columns:
    print(f"   💰 Користувачі з депозитами: {matched_df['has_deposit'].sum():,}")
    print(f"   🎯 Конверсія push→депозит: {matched_df['has_deposit'].mean()*100:.2f}%")
if 'total_revenue' in matched_df.columns:
    print(f"   💵 Загальний дохід: ${matched_df['total_revenue'].sum():,.2f}")
    print(f"   💵 ARPU: ${matched_df['total_revenue'].sum()/len(matched_df):.2f}")

print(f"\n✅ МАТЧИНГ ЗАВЕРШЕНО УСПІШНО!")
print(f"📂 Файли збережено в data/processed/ та outputs/tables/")
print(f"➡️ Готово для наступних етапів аналізу")


🔗 КРОК 6: ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ
----------------------------------------
✅ Основний файл: matched_push_conversions.parquet
✅ Файли для A/B аналізу збережено
✅ Файли для гео аналізу збережено
✅ Резюме матчингу збережено: matching_summary.json

📈 ФІНАЛЬНА СТАТИСТИКА МАТЧИНГУ:
   👥 Загалом користувачів: 3,219,484
   🔗 Спільних GADID: 109,858
   📈 Якість матчингу: excellent
   💰 Користувачі з депозитами: 41,193
   🎯 Конверсія push→депозит: 1.28%
   💵 Загальний дохід: $226,815.28
   💵 ARPU: $0.07

✅ МАТЧИНГ ЗАВЕРШЕНО УСПІШНО!
📂 Файли збережено в data/processed/ та outputs/tables/
➡️ Готово для наступних етапів аналізу


## **КРОК 7: Підготовка до наступних етапів**

In [8]:
print("\n🔗 КРОК 7: ПІДГОТОВКА ДО НАСТУПНИХ ЕТАПІВ")
print("-" * 40)

print("📁 СТВОРЕНІ ФАЙЛИ:")
print("   📊 matched_push_conversions.parquet - основний файл матчингу")
print("   🏷️ ab_analysis_ready.* - готові дані для A/B аналізу")
print("   🌍 geo_analysis_ready.* - готові дані для гео аналізу")
print("   📋 matching_summary.json - підсумкова статистика")

print("\n🎯 ГОТОВНІСТЬ ДО НАСТУПНИХ НОУТБУКІВ:")

# Перевіряємо готовність для A/B аналізу
if 'ab_group' in matched_df.columns and matched_df['ab_group'].nunique() > 1:
    print(f"   ✅ 04_ab_analysis.ipynb - готово ({matched_df['ab_group'].nunique()} A/B груп)")
else:
    print(f"   ⚠️ 04_ab_analysis.ipynb - немає A/B груп для аналізу")

# Перевіряємо готовність для гео аналізу
if 'tier_final' in matched_df.columns and matched_df['tier_final'].nunique() > 1:
    tier_counts = matched_df['tier_final'].value_counts()
    print(f"   ✅ 05_geo_tier_analysis.ipynb - готово ({tier_counts.to_dict()})")
else:
    print(f"   ⚠️ 05_geo_tier_analysis.ipynb - немає tier даних")

# Завжди готово для фінального звіту
print(f"   ✅ 06_final_report.ipynb - готово для фінального звіту")

print("\n💡 РЕКОМЕНДАЦІЇ:")
if len(intersection) > 10000:
    print("   🎉 Відмінна якість матчингу - можна проводити поглиблений аналіз")
elif len(intersection) > 1000:
    print("   👍 Хороша якість матчингу - результати будуть статистично значущими")
else:
    print("   ⚠️ Низька якість матчингу - обережно з висновками")

if 'has_deposit' in matched_df.columns:
    conv_rate = matched_df['has_deposit'].mean() * 100
    if conv_rate > 5:
        print(f"   💰 Високий коефіцієнт конверсії ({conv_rate:.1f}%) - гарні можливості для аналізу")
    elif conv_rate > 1:
        print(f"   💰 Помірний коефіцієнт конверсії ({conv_rate:.1f}%) - достатньо для аналізу")
    else:
        print(f"   💰 Низький коефіцієнт конверсії ({conv_rate:.1f}%) - потрібен більший обсяг даних")

print(f"\n🚀 СИСТЕМА ГОТОВА ДО ПОДАЛЬШОГО АНАЛІЗУ!")


🔗 КРОК 7: ПІДГОТОВКА ДО НАСТУПНИХ ЕТАПІВ
----------------------------------------
📁 СТВОРЕНІ ФАЙЛИ:
   📊 matched_push_conversions.parquet - основний файл матчингу
   🏷️ ab_analysis_ready.* - готові дані для A/B аналізу
   🌍 geo_analysis_ready.* - готові дані для гео аналізу
   📋 matching_summary.json - підсумкова статистика

🎯 ГОТОВНІСТЬ ДО НАСТУПНИХ НОУТБУКІВ:
   ✅ 04_ab_analysis.ipynb - готово (6 A/B груп)
   ✅ 05_geo_tier_analysis.ipynb - готово ({'Unknown': 1750835, 'Tier 2': 692620, 'Other': 317990, 'Tier 3': 311368, 'Tier 1': 146671})
   ✅ 06_final_report.ipynb - готово для фінального звіту

💡 РЕКОМЕНДАЦІЇ:
   🎉 Відмінна якість матчингу - можна проводити поглиблений аналіз
   💰 Помірний коефіцієнт конверсії (1.3%) - достатньо для аналізу

🚀 СИСТЕМА ГОТОВА ДО ПОДАЛЬШОГО АНАЛІЗУ!
